In [1]:
from mysql.connector import MySQLConnection
from contextlib import closing
import pandas as pd
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
dbconfig = {
    'user': os.getenv('MYSQL_USER'),
    'host': 'localhost',
    'port': '3306',
    'database': 'LittleLemonDB',
    'password': os.getenv('MYSQL_PASSWORD')
}

In [4]:
def execute_dmc(query):
    with closing(
        MySQLConnection(**dbconfig)
    ) as connection:
        with closing(
            connection.cursor()
        ) as cursor:
            cursor.execute(
                query
            )
            connection.commit()

In [5]:
def execute_dql(query):
    with closing(
        MySQLConnection(**dbconfig)
    ) as connection:
        with closing(
            connection.cursor()
        ) as cursor:
            cursor.execute(query)
            return pd.DataFrame(
                data=cursor.fetchall(),
                columns=cursor.column_names
            )

In [6]:
execute_dql(
    '''
    SELECT * FROM Orders;
    '''
)

,OrderID,OrderID_Not_Key,RowNumber,OrderDate,Quantity,TotalCost,CustomerID,MenuMenuItemsID
0,1,54-366-6861,1,2020-06-15,2,125,72-055-7985,1
1,2,87-599-0366,13,2020-12-03,2,93,09-688-8521,1
2,3,97-507-9861,121,2020-12-24,2,93,86-200-6226,1
3,4,43-009-8040,133,2020-01-11,2,320,50-716-5754,1
4,5,06-934-9396,229,2020-09-30,2,144,14-968-9727,1
...,...,...,...,...,...,...,...,...
20995,20996,53-322-3671,20671,2021-01-13,2,92,13-040-2063,54
20996,20997,62-201-9287,20683,2019-06-26,3,310,05-480-9942,54
20997,20998,26-802-1005,20791,2020-09-01,2,320,53-704-6388,54
20998,20999,73-375-8634,20899,2020-05-31,3,75,71-750-9970,54


## Exercise 1

### Task 1 - Creating an Order View. 

In [9]:
execute_dmc(
    '''
    CREATE OR REPLACE VIEW OrdersView AS
    SELECT OrderID, Quantity, TotalCost
    FROM Orders
    WHERE Quantity > 2;
    '''
)

In [10]:
execute_dql(
    '''
    SELECT * FROM OrdersView;
    '''
)

,OrderID,Quantity,TotalCost
0,673,3,75
1,675,3,92
2,677,3,320
3,679,3,125
4,681,3,168
...,...,...,...
7870,20991,3,144
7871,20993,3,190
7872,20995,3,52
7873,20997,3,310


### Task 2 - Query to find all orders with cost > 150

In [19]:
execute_dql(
    '''
    SELECT CustomerID, CustomerName, OrderID, TotalCost, MainCourse
    FROM Customers
    INNER JOIN Orders
    USING (CustomerID)
    INNER JOIN MenuMenuItems
    USING (MenuMenuItemsID)
    INNER JOIN MenuItems
    USING (MenuItemsID)
    WHERE TotalCost > 150;
    '''
)

,CustomerID,CustomerName,OrderID,TotalCost,MainCourse
0,50-716-5754,Granville Mallison,4,320,Greek salad
1,81-541-0260,Ginevra Pally,10,168,Greek salad
2,97-850-3306,Cyril Copelli,11,190,Greek salad
3,61-676-0339,Barnabas Casson,13,310,Greek salad
4,30-825-3298,Doug Logg,16,310,Greek salad
...,...,...,...,...,...
7635,80-680-2541,Sue Fargie,20988,320,Kabasa
7636,17-062-2228,Hale Danilchik,20992,168,Kabasa
7637,37-335-7316,Elliott Avard,20993,190,Kabasa
7638,05-480-9942,Sabina Corneille,20997,310,Kabasa


### Task 3 - Find MenuIDs with more than a quantity of 2

In [22]:
execute_dql(
    '''
    SELECT MainCourse
    FROM MenuItems
    INNER JOIN MenuMenuItems
    USING (MenuItemsID)
    WHERE MenuMenuItemsID = ANY (
        SELECT MenuMenuItemsID
        FROM Orders
        WHERE Orders.Quantity > 2
    );
    '''
)

,MainCourse
0,Greek salad
1,Bean soup
2,Pizza
3,Carbonara
4,Kabasa
5,Shwarma
6,Bean soup
7,Pizza
8,Carbonara
9,Kabasa


## Exercise 2

### Task 1

In [23]:
execute_dmc(
    '''
    DROP PROCEDURE IF EXISTS GetMaxQuantity;
    '''
)

In [24]:
execute_dmc(
    '''
    CREATE PROCEDURE GetMaxQuantity()
    BEGIN
        SELECT Quantity
        FROM Orders
        ORDER BY Quantity DESC
        LIMIT 1;
    END
    '''
)

In [40]:
with closing(
    MySQLConnection(**dbconfig)
) as connection:
    with closing(
        connection.cursor()
    ) as cursor:
        cursor.callproc('GetMaxQuantity', ())
        print(
            pd.DataFrame(
                data=(x:=next(cursor.stored_results())),
                columns=x.column_names
            )
        )

   Quantity
0         3


### Task 3

In [48]:
execute_dmc(
    '''
    DROP PROCEDURE IF EXISTS CancelOrder;
    '''
)

In [49]:
execute_dmc(
    '''
    CREATE PROCEDURE CancelOrder(IN OrderIDvar INT)
    BEGIN
        DELETE FROM Orders
        WHERE OrderID = OrderIDvar;
    END
    '''
)

In [52]:
with closing(
    MySQLConnection(**dbconfig)
) as connection:
    with closing(
        connection.cursor()
    ) as cursor:
        cursor.callproc('CancelOrder', (1,))
        cursor.execute(
            'SELECT * FROM Orders'
        )
        print(
            pd.DataFrame(
                data=cursor.fetchall(),
                columns=cursor.column_names
            )
        )

StopIteration: 